### Linear Regression -- Our Goals

* Remind ourselves what LR is and what we're trying to do
* Define the LR problem for our particular data set
* Identify our X and y variables
* Use pandas to run a regression
* Assess the quality of the model

We'll be working with a data set from Capital Bikeshare that was used in a [Kaggle competition](https://www.kaggle.com/c/bike-sharing-demand/data).

The objective of the competition is to predict total ridership of Capital Bikeshare in any given hour.

##### Data Dictionary (from Kaggle)

| Variable| Description |
|---------|----------------|
|datetime| hourly date + timestamp  |
|season|  1=winter, 2=spring, 3=summer, 4=fall |
|holiday| whether the day is considered a holiday|
|workingday| whether the day is neither a weekend nor holiday|
|weather| 1 -> Clear or partly cloudy;  2 -> Clouds and mist; 3 -> Light rain or snow;  4 -> Heavy rain or snow|
|temp| temperature in Celsius|
|atemp| "feels like" temperature in Celsius|
|humidity| relative humidity|
|windspeed| wind speed|
|casual| number of non-registered user rentals initiated|
|registered| number of registered user rentals initiated|
|count| number of total rentals|


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 14
plt.style.use("fivethirtyeight")

<a id="read-in-the--capital-bikeshare-data"></a>
### Read In the Capital Bikeshare Data

In [ ]:
# Read the data and set the datetime as the index.
#  -- We sometimes want to do this if (a) that column is unique
#       and acts as an ID, and (b) we don't want to use it as a
#       predictor
#  -- Notice also forcing it to be a real date/time instead of string
bikes = pd.read_csv('bikeshare.csv', index_col='datetime', parse_dates=True)

In [ ]:
bikes.info()

In [ ]:
# Preview the first five rows of the DataFrame.
bikes.head(5)

In [ ]:
bikes.tail(5)

<span style="color:blue;font-size:120%">What does each observation represent?</span>

<span style="color:blue;font-size:120%">What is the response variable (as defined by Kaggle)?</span>

<span style="color:blue;font-size:120%">How many features are there that could be predictors in a regression?  What are other potential features?</span>

#### Rename some variables

"count" is a method in Pandas (and a very non-specific name), so it's best to name that column something else

In general, you may want to rename columns if it is not obvious what might be stored in them, and you might also adopt the convention of including a column's units or type in its name. Although we will only rename the target column here, a few examples might be to rename:

| old name | new name |
| ---    | --- |
| temp | temp_celsius
| windspeed | windspeed_knots
| casual | num_casual_users
| registered | num_registered_users
| season | season_num
| holiday | is_holiday
| workingday | is_workingday
| humidity | humidity_percent

This names should make it obvious what is stored in each column. The downside is slightly longer column names, which could affect table readability in Jupyter. It would be ideal to use very specific names in CSV files to assist others reading them. In your own code, use whatever makes sense for your work -- if you are viewing lots of Pandas tables, you may want to use shorter names. However, readable specific names are preferred in Python code since it prevents mistakes and makes your work more clear to readers

<span style="color:blue;font-size:120%">Use the .rename() method to rename count to total_rentals</span>

<a id="visualizing-the-data"></a>
### Visualizing the Data

It is important to have a general feeling for what the data looks like before building a model. Ideally, before creating the model you would have some sense of which variables might matter most to predict the response. This dataset is fairly intuitive (and the purpose of this lesson is not visualization), so we will keep the visualization short.

In [ ]:
sns.pairplot(bikes);

We are going to focus on the single variate ```temp``` first

In [ ]:
bikes.atemp.plot(kind="box")
bikes.atemp.describe()

In [ ]:
bikes.atemp.hist(bins=100)

In [ ]:
# Pandas scatterplot
bikes.plot(kind='scatter', x='temp', y='total_rentals', alpha=0.2);

In [ ]:
# Seaborn scatterplot with regression line
#    aspect => plot is twice as long as it is high
#    alpha => transparancy on scale of 0 (transparent) to 1 (opaque)

sns.lmplot(x='temp', y='total_rentals', data=bikes, aspect = 2.0, scatter_kws={'alpha':0.2});

#### Linear Regression Basics
---

<a id="form-of-linear-regression"></a>
### Form of Linear Regression

Recall that each model always contains some amount of random irreducible error $\epsilon$. So, given a prediction $\hat{y}$, the actual $y = \hat{y} + \epsilon$. Below, we will assume $y$ is exactly linear.

- We are often taught the formula for a line is: $y = mx + b$.
- But it's more common in the literature to use Greek letters for coefficients: $y = \alpha + \beta X$.

---

Here, we will generalize this to $n$ independent variables as follows:

$$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n + \epsilon\quad\quad =\quad\quad (\sum_{i=0}^{n} \beta_ix_i) + \epsilon$$

- $y$ is the response.
- $\beta_0$ is the intercept.
- $\beta_1$ is the coefficient for $x_1$ (the first feature).
- $\beta_n$ is the coefficient for $x_n$ (the nth feature).
- $\epsilon$ is the _error_ term which is independent of x, y, $i$, and $n$

A practical example of this applied to our data might be:

$total\_rides = 20 + -2 \cdot temp + -3 \cdot windspeed\ +\ ...\ +\ 0.1 \cdot registered$

This equation is still called **linear** because the highest degree of the independent variables (e.g. $x_i$) is 1. Note that because the $\beta$ values are constants, they will not be independent variables in the final model, as seen above.


<span style="color:blue; font-size:120%"> What are the limits of this linearity assumption?  What kind of relationships can't we capture?  Can you think of real examples where linearity is violated?</span>

*  Non-linear such as squared or exponential
* Interrelationship terms -- X1 * X2
* Definitional relationships   X1 AND X2
* Contribution of X to Y changes with the value of Y -- for example humidity affects ridership more on hot days than on cold

---

In the regression equation
$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n + \epsilon$

the $\beta$ values are called the **model coefficients**:

- These values are estimated (or "learned") during the model fitting process using the **least squares criterion**.
- Specifically, we are trying to find the line (mathematically) that minimizes the **sum of squared residuals** (or "sum of squared errors").
- Once we've learned these coefficients, we can use the model to predict the response.

![Estimating coefficients](estimating_coefficients.png)

Remember the book defines
$$MSE = \frac{1} {n} \| \hat{y}(\mathbf{X}) - \vec{y} \|$$

and said the algorithm chooses coefficients that minimize this quantity. Same concept with different notation.  (Also if you are minimizing a quantity $E$ the same value will minimize $E/2$.)

In the diagram above:

- The black dots are the **observed values** of x and y.
- The blue line is our **least squares line**.
- The red lines are the **residuals**, which are the vertical distances between the observed values and the least squares line.

If there are two predictors, the model fits a plane, and residuals are the distance between the observed value and the least squares plane.

![Regression with Two Variates](multiple_regression_plane.png)

If there are more than two predictors, it's hard to visualize.

---
Linear regression is an example of "supervised learning" -- this is learning some model on the basis of *training examples* that are *labeled* with the correct outcome value. 
In our example, our $i^{th}$ training example looks like this mathematically

$$(x_{i,1} x_{i,2}, ..., x_{i,n}, y)$$

but sometimes the $x$ and $y$ are represented separately and in matrix notation look like this for a situation where we have $m$ independent variables and $n$ observations

$$
\mathbf{X} = 
\begin{bmatrix}
x_{1,1} & x_{1,2}, \ldots, & x_{1, m}\\
x_{2,1} & x_{2,2}, \ldots, & x_{2, m}\\
\ldots\\
x_{n,1} & x_{n,2}, \ldots, & x_{n, m}
\end{bmatrix}
\quad\quad\quad
\mathbf{Y} = 
\begin{bmatrix}
y_1\\
y_2\\
\ldots\\
y_n
\end{bmatrix}
$$


<a id="requirements-for-working-with-data-in-scikit-learn"></a>
### Requirements for Working With Data in scikit-learn

1. Features and response should be separate objects.
2. Features and response should be entirely numeric.
3. Features and response should be NumPy arrays (or easily converted to NumPy arrays).
4. Features and response should have specific shapes (outlined below).

<a id="building-a-linear-regression-model-in-sklearn"></a>
### Building a (Single) Linear Regression Model in sklearn

#### Create a feature matrix called X that holds a `DataFrame` with only the temp variable and a `Series` called y that has the "total_rentals" column.

In [ ]:
# Create X and y.
feature_cols = ['temp']
X = bikes[feature_cols]
y = bikes.total_rentals

<span style="color:blue; font-size:120%">Why do you think X is capitalized but y is not?</span>

In [ ]:
print(type(X))
print(type(y))
print(X.shape)
print(y.shape)

<a id="scikit-learns--step-modeling-pattern"></a>
### scikit-learn's Four-Step Modeling Pattern

**Step 1:** Import the class you plan to use.

In [ ]:
from sklearn.linear_model import LinearRegression

**Step 2:** *Instantiate* the *estimator.*

- Estimator is scikit-learn's term for model.
- Instantiate is an object-oriented programming term meaning "make an instance of"

In [ ]:
# Make an instance of a LinearRegression object.
lr = LinearRegression()
type(lr)

- Created an object that "knows" how to do linear regression, and is just waiting for data.
- Name of the object does not matter.
- All parameters not specified are set to their defaults.
- Can specify tuning parameters (aka "hyperparameters") during this step. 

To view the possible parameters, either use the `help` built-in function or evaluate the newly instantiated model, as follows:

In [ ]:
lr

**Step 3:** Fit the model with data (aka "model training").

- Model is "learning" the relationship between X and y in our "training data."
- Process through which learning occurs varies by model.
- Occurs in-place.

In [ ]:
lr.fit(X, y)

- Once a model has been fit with data, it's called a "fitted model."

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

**Step 4:** Predict the response for a new observation.

- New observations are called "out-of-sample" data.
- Uses the information it learned during the model training process.

In [ ]:
# If we predict at x=0 we will get the intercept
lr.predict(np.array([0]).reshape(-1,1))
lr.predict([[0]])

In [ ]:
lr.predict(np.array([0]).reshape(1,-1))

In [ ]:
#  Just for convenience -- take a number (x) gets its predicted y
def lrpred(xval):
    return lr.predict(np.array([xval]).reshape(1,-1))[0]


In [ ]:
print(lrpred(4) - lrpred(3))
print(lrpred(21) - lrpred(20))

In [ ]:
# Predict multiple x values at once

X_new = [[4], [3],[21],[20]]
lr.predict(X_new)

Let's ask the model to make three predictions, at temp values 0, 10, 100. To do this, our feature matrix is always a 2-D array where each row is a list of features. Since we only have a single feature, the temperature, each row will contain only a single value.

In [ ]:
lr.predict([[0], [10], [100]])


What we just predicted using our model is, "If the temperature is 0 degrees, the total number of bike rentals will be ~6.046, and if the temperature is 10 degrees the total number of bike rentals will ~97.751."

<span style="color:blue;font-size=120%">How would you informally confirm that these are reasonable predictions?</span>

* Make sure they obey the regression equation
* Look for observations "close" to them in the data set and see if the rentals value is about the same

Interpreting the intercept ($\beta_0$):

- It is the value of $y$ when all independent variables are 0.
- Here, it is the estimated number of rentals when the temperature is 0 degrees Celsius.
- <span style="color:blue">**Note:** It does not always make sense to interpret the intercept. (Why?)</span>

Interpreting the "temp" coefficient ($\beta_1$):

- **Interpretation:** An increase of 1 degree Celcius is _associated with_ increasing the number of total rentals by $\beta_1$.
- Here, a temperature increase of 1 degree Celsius is _associated with_ a rental increase of 9.17 bikes.
- This is not a statement of causation.
- $\beta_1$ would be **negative** if an increase in temperature was associated with a **decrease** in total rentals.
- $\beta_1$ would be **zero** if temperature is not associated with total rentals.

<span style="color:blue; font-size:120%">How many bike rentals would we predict if the temperature was 25 degrees Celsius?  Do the calculation both manually, and using the model.</span>

In [ ]:
# Manually calculate the prediction.
linreg.intercept_  +   linreg.coef_ * [25]

In [ ]:
lr.predict([[25]])

<a id="does-the-scale-of-the-features-matter"></a>
### Does the Scale of the Features Matter?

Let's say that temperature was measured in Fahrenheit, rather than Celsius. How would that affect the model?

In [ ]:
# Create a new column for Fahrenheit temperature.
bikes['temp_F'] = bikes.temp * 1.8 + 32
bikes.head()

In [ ]:
# Seaborn scatterplot with regression line
sns.lmplot(x='temp_F', y='total_rentals', data=bikes, aspect=1.5, scatter_kws={'alpha':0.2});

In [ ]:
sns.lmplot(x='temp', y='total_rentals', data=bikes, aspect=1.5, scatter_kws={'alpha':0.2});

#### Rebuild the `LinearRegression` from above using the `temp_F` features instead.

In [ ]:
# Create X and y.
feature_cols = ['temp_F']
X = bikes[feature_cols]
y = bikes.total_rentals

# Instantiate and fit.
linreg = LinearRegression()
linreg.fit(X, y)

# Print the coefficients.
print(linreg.intercept_)
print(linreg.coef_)

#### Convert 25 degrees Celsius to Fahrenheit.

In [ ]:
25 * 1.8 + 32

#### Predict rentals for 77 degrees Fahrenheit.

In [ ]:
linreg.predict(77)

**Conclusion:** The scale of the features is irrelevant for prediction in linear regression models. When changing the scale, we simply change our interpretation of the coefficients.  

In [ ]:
# Remove the temp_F column.
bikes.drop('temp_F', axis=1, inplace=True)

##### Work With Multiple Features
---

We've demonstrated simple linear regression with one feature to gain an intuition, but the benefit of modeling is the ability to reason about hundreds of features at once. There is no limit to the number of features you can use. However, often a small set of features accounts for most of the variance (assuming there is a linear relationship at all). We will start by using four features.

<a id="visualizing-the-data-part-"></a>
### Visualizing the Data (Part 2)

#### Explore more features.

In [ ]:
# Create feature column variables
feature_cols = ['temp', 'season', 'weather', 'humidity']

#### Create a subset of scatterplot matrix using Seaborn.
We can use pairplot with the y_vars argument to only show relationships with the `total_rentals` variable

In [ ]:
# multiple scatterplots in Seaborn
sns.pairplot(bikes, x_vars=feature_cols, y_vars='total_rentals', kind='reg');

#### Recreate the same functionality using Pandas.

In [ ]:
# Multiple scatterplots in Pandas -- put plots on a 1x4 grid, and force the same Y axis
fig, axs = plt.subplots(1, len(feature_cols), sharey=True)
for index, feature in enumerate(feature_cols):
    bikes.plot(kind='scatter', x=feature, y='total_rentals', ax=axs[index], figsize=(16, 3))

#### Are you seeing anything you didn't expect?

#### Explore the season variable using a cross-tab.

In [ ]:
# Cross-tabulation of season and month
pd.crosstab(bikes.season, bikes.index.month)

#### Explore the season variable using a box plot.

In [ ]:
# Box plot of rentals, grouped by season
bikes.boxplot(column='total_rentals', by='season');

Notably:

- A line can't capture a nonlinear relationship.
- The problem is not really with nonlinearity -- *season* is probably better coded as a "categorical variable" -- a variable that takes one value from a set of values (like temperatures = {hot, medium, cold}).  There are special techniques for dealing with categorical variables, more on that later.

#### Look at rentals over time.

In [ ]:
# Line plot of rentals
bikes.total_rentals.plot();

#### What does this tell us?

There are more rentals in the winter than the spring, but only because the system is experiencing overall growth and the winter months happen to come after the spring months.

So does overall growth completely explain the model?

In [ ]:
df = pd.DataFrame(bikes['total_rentals'])
df['hours_since_open'] = range(0, bikes.shape[0])
sns.lmplot(x='hours_since_open', y='total_rentals', data=df, aspect=1.5, scatter_kws={'alpha':0.2});

Question for later: how much better -- if at all -- is any model we build from the column featues than this model?

#### Look at the correlation matrix for the bikes `DataFrame`.

In [ ]:
# Correlation matrix (ranges from 1 to -1)
bikes.corr()

#### Use a heat map to make it easier to read the correlation matrix.

In [ ]:
# Visualize correlation matrix in Seaborn using a heat map.
sns.heatmap(bikes.corr())

<span style="color:blue;font-size=120%">What relationships do you notice and which are important for our model building?</span>

##### Adding More Features to the Model

In the previous example, one variable explained the variance of another; however, more often than not, we will need multiple variables. 

- For example, a house's price may be best measured by square feet, but a lot of other variables play a vital role: bedrooms, bathrooms, location, appliances, etc. 

- For a linear regression, we want these variables to be largely independent of one another, but all of them should help explain the y variable.

- Eliminating "colinear" variables is a separate topic.  For now let's realize that temp and atemp are linearly related so we don't need both
- Especially we have to eliminate attributes that are "definitional" of the dependent variable!  So registered and casual have to go, but maybe the ratio of registered to total users would be useful?

#### Create another `LinearRegression` instance that is fit using temp, season, weather, and humidity.

In [ ]:
# Create a list of features.
feature_cols = ['temp', 'season', 'weather', 'humidity']

In [ ]:
# Create X and y.
X = bikes[feature_cols]
y = bikes.total_rentals

# Instantiate and fit.
linreg = LinearRegression()
linreg.fit(X, y)

# Print the coefficients.
print(linreg.intercept_)
print(linreg.coef_)

#### Display the linear regression coefficient along with the feature names.

In [ ]:
# Pair the feature names with the coefficients.
list(zip(feature_cols, linreg.coef_))

Interpreting the coefficients:

- Holding all other features fixed, a 1-unit increase in temperature is associated with a rental increase of 7.86 bikes.
- Holding all other features fixed, a 1-unit increase in season is associated with a rental increase of 22.5 bikes.
- Holding all other features fixed, a 1-unit increase in weather is associated with a rental increase of 6.67 bikes.
- Holding all other features fixed, a 1-unit increase in humidity is associated with a rental decrease of 3.12 bikes.

Does anything look incorrect and does not reflect reality?

##### Evaluating Models / Comparing Models
---

We can make linear models now, but how do we select the "best" model to use for our applications? We can answer questions about how well a model fits the observed data, but the definition of "best" can depend on other business factors (like cost associated with the model making errors).

Two different questions
1.  How well does the model fit the observed data (training set)
2.  How well does will the model predict future observations in production.

Second may differ from the first because
1.  The training set was too small -- important features in the population didn't show up enough to be noticed
2.  The model found spurious signal in the training set "total sales is higher on days that begin with 'T'." (Overfitting)

Looking at the first:
* The $R^2$ statistic -- the amount the model reduces variability
* Significance tests on the slope coefficient(s) -- if a variable's coefficient is 0, then it is not adding predictive power


### Statistics from the Residuals

**Residual sum of squares**
$$RSS = \sum_{i=1}^{n} (y_i - \hat{y})^2$$

This is familiar -- how much each prediction deviates from the observed. Remember, the regression chooses coefficients and intercept to minimize this value ($MSE$)

**Residual standard error**
$$RSE = \sqrt{ \frac{1}{n-2} RSS} $$
This is the "average deviation" of a prediction from the actual value.  It is in terms of $y$ units (e.g. total rentals) so comparing models can be tricky

Remember, there is variance in the observations themselves apart from any model;  if the data itself has high variance, then a predictive model will have high variance too.

**Total sum of squares**
$$TSS = \sum_{i=1}^n(y_i - \overline{y})^2$$
Standard definition of deviance of observations from the mean -- model independent.

**R-squared is proportion of the total variance accounted for by the model**
$$R^2 \quad=\quad \frac{(TSS - RSS)}{TSS} \quad = \quad 1 - \frac{RSS}{TSS}$$
If RSS = 0 then $R^2$ = 1 which means RSS accounts for 100% of the variance (it perfectly predicts the observed).  If $RSS = TSS$ then the regression does not reduce any variance (worst case model), same as guessing at the mean.

$R^2$ is a ratio (unit-free) so can be compared across models, but not clear what a "good enough" value is.  At least partially depends on what we know about the underlying system -- contributors to low values might be
* Underlying system is not linear -- so linear model will no do well
* Underlying system is linear, but we are missing key variates

**Adjusted R-squared adjusts for number of variables in the model**
$R^2$ increases as more predictors are added to the model, regardless of whether they improve prediction.
The adjusted version corrects for this
$$ {R}^{2}_{adj \quad}= \quad 1-\left[\frac{(1-R^{2})(n-1)}{n-p-1}\right]$$
where $n$ is the number of observations and $p$ is the number of variables in the model

In [ ]:
def tss(actual):
    return ((actual - actual.mean())**2).sum()

def rss(actual, predicted):
    return ((actual - predicted)**2).sum()

def rsquared(actual, predicted):
    return (tss(actual) - rss(actual, predicted))/tss(actual)

def adjust_rsquared(rquared, n, p):
    return 1 - ((1-rsquared) * (n - 1) /(n - p - 1))


#### This is a standard summary from a regression.  We will look at only a couple of pieces

In [ ]:
import statsmodels.api as sm
bikes['rand'] = 100 + 20*np.random.rand(bikes.shape[0])
X = sm.add_constant(bikes[['atemp', 'windspeed', 'season', 'rand']])
y = bikes['total_rentals']
results = sm.OLS(y, X).fit()
print(results.summary())

**Highlights of the Regression Results Table**
* The F-statistic tests the hypothesis "all of the coefficients are 0" -- it's a ratio, and a value of 1 indicates no relationship.  The F-statistic strongly indicates that the model is better at predicting total rentals than using the mean value
* For the intercept and coefficients, the $p$ value tests the hypothesis that the parameter is 0.  Values close 0 mean the parameter is unlikely to be 0.  Notice the confidence interval and $p$ value for *rand* -- its coefficient is very likely to be 0. 

In [ ]:
#  A way to get some statistics like adjusted R-squared for a set of columns
def statsmodel_summary(dataset, columns):
    X = sm.add_constant(dataset[columns])
    Y = dataset['total_rentals']
    results = sm.OLS(y, X).fit()
    print(results.summary())

In [ ]:
# For example, this took away the random column -- notice R-squared stayed the same
# but adjusted R-squared went up a little.

statsmodel_summary(bikes, ['atemp', 'windspeed', 'season'])

<a id="handling-categorical-features"></a>
### Handling Categorical Features

scikit-learn expects all features to be numeric. So how do we include a categorical feature in our model?

- **Ordered categories:** Transform them to sensible numeric values (example: small=1, medium=2, large=3)
- **Unordered categories:** Use dummy encoding (0/1). Here, each possible category would become a separate feature.

What are the categorical features in our data set?

- **Ordered categories:** `weather` (already encoded with sensible numeric values)
- **Unordered categories:** `season` (needs dummy encoding), `holiday` (already dummy encoded), `workingday` (already dummy encoded)

For season, we can't simply leave the encoding as 1 = spring, 2 = summer, 3 = fall, and 4 = winter, because that would imply an ordered relationship. Instead, we create multiple dummy variables.

#### Create dummy variables using `get_dummies` from Pandas.

In [ ]:
season_dummies = pd.get_dummies(bikes.season, prefix='season')

#### Inspect the `DataFrame` of `dummies`.

In [ ]:
# Print five random rows.
season_dummies.sample(n=5, random_state=1)

However, we actually only need three dummy variables (not four), and thus we'll drop the first dummy variable.

Why? Because three dummies captures all of the "information" about the season feature, and implicitly defines spring (season 1) as the baseline level.

This circles back to the concept multicollinearity, except instead of one feature being highly correlated to another, the information gained from three features is directly correlated to the fourth.

#### Drop the first column.

In [ ]:
season_dummies.drop(season_dummies.columns[0], axis=1, inplace=True)

#### Reinspect the `DataFrame` of `dummies`.

In [ ]:
# Print five random rows.
season_dummies.sample(n=5, random_state=1)

In general, if you have a categorical feature with k possible values, you create k-1 dummy variables.

If that's confusing, think about why we only need one dummy variable for `holiday`, not two dummy variables (`holiday_yes` and `holiday_no`).

#### We now need to concatenate the two `DataFrames` together.

In [ ]:
# Concatenate the original DataFrame and the dummy DataFrame (axis=0 means rows, axis=1 means columns).
bikes_dummies = pd.concat([bikes, season_dummies], axis=1)

# Print 5 random rows.
bikes_dummies.sample(n=5, random_state=1)

#### Rerun the linear regression with dummy variables included.

In [ ]:
# Include dummy variables for season in the model.
feature_cols = ['atemp', 'windspeed', 'season_2', 'season_3', 'season_4']
X = bikes_dummies[feature_cols]
y = bikes_dummies.total_rentals

linreg = LinearRegression()
linreg.fit(X, y)

list(zip(feature_cols, linreg.coef_))

In [ ]:
#  Same model with dummy variables -- R-squared and adjusted R-squared both improve
statsmodel_summary(bikes_dummies, ['atemp', 'windspeed', 'season_2', 'season_3', 'season_4'])

How do we interpret the season coefficients? They are measured against the baseline (spring):

- Holding all other features fixed, summer is associated with a rental decrease of 3.39 bikes compared to the spring.
- Holding all other features fixed, fall is associated with a rental decrease of 41.7 bikes compared to the spring.
- Holding all other features fixed, winter is associated with a rental increase of 64.4 bikes compared to the spring.

Would it matter if we changed which season was defined as the baseline?

- No, it would simply change our interpretation of the coefficients.

In most situations, it is best to have the dummy that is your baseline be the category that has the largest representation.

**Important:** Dummy encoding is relevant for all machine learning models, not just linear regression models.

Compare adjusted R-squared for various models

In [ ]:
#  This is worst case scenario -- rand should have no predictive value
statsmodel_summary(bikes, ['rand'])

In [ ]:
# This is the full model -- we see an improvement -- only question is whether we could remove certain 
#  variables and improve adjusted R-squared.  Not likely since the "penalty" for adding a variable
#  seems to be low
full_model_columns = ['weather', 'season_2', 'season_3', 'season_4', 'holiday', 'workingday', 'windspeed', 'temp', 'humidity']
statsmodel_summary(bikes_dummies, full_model_columns)

##### Next Step -- Feature Engineering
* Choose additional features, that *might* be relevant.  For example, hours since opening, hour of the day (extracted from timestamp)
* Look for interaction effects -- for example, the influence of temperature on rentals might depend on the season

No good objective methodology for doing that, but evaluation is always to compare the model with the variable(s) to the model without them, evaluate on the basis of Adjusted R^2 or other metric.

##### Summary:  Comparing Linear Regression With Other Models

Advantages of linear regression:

- Simple to explain.
- Highly interpretable.
- Model training and prediction are fast.
- No tuning is required (excluding regularization).
- Features don't need scaling.
- Can perform well with a small number of observations.
- Well understood.

Disadvantages of linear regression:

- Presumes a linear relationship between the features and the response.
- Performance is (generally) not competitive with the best supervised learning methods due to high bias.
- Can't automatically learn feature interactions.